In [ ]:
# Adam Brenner
# Assignment 5

In [1]:
import folium
from folium.plugins import MarkerCluster
from shapely.geometry import Polygon, mapping
import plotly.graph_objects as go
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import geojson
import plotly.express as px

In [2]:
# PART 1

# Read in Capitals CSV and put the latitudes and longitudes into a list
capData = pd.read_csv("capitals_lat_lon.csv", sep = "\\t", engine = 'python')

loc = capData[['Latitude', 'Longitude']]
locations = loc.values.tolist()

In [3]:
# Create folium map and add marker clusters to make the map more accessible.
# Add a marker for every capital in the dataset
capMap = folium.Map()

capMapCluster = MarkerCluster().add_to(capMap)

for point in range(0, len(locations)):
    folium.Marker(locations[point], popup = capData['Capital'][point],
                  tooltip = capData['Capital'][point]).add_to(capMapCluster)
    
capMap.save("capitals_map.html")

capMap

In [5]:
# PART 2

# 2 Estimations for drawing a triangle on as much of Africa as possible

afrUL = [27.941206126054926, -12.931072059079822] # Port Tarfaya
afrLT = [-33.786245090221236, 25.368248744591927] # Port Elizabeth
afrUR = [31.277621642684554, 32.318931717753586] # Port Said

afrUL2 = [35.762418187520375, -5.8334422185585] # Tangier
afrLT2 = [-33.92493006610948, 18.422301377595126] # Cape Town
afrUR2 = [9.499420624096508, 50.81222392582046] # Bandarbeyla

In [7]:
# Function to find the center of said triangles from above
def centroid(l1, l2, l3):
    ox = (l1[0] + l2[0] + l3[0]) / 3
    oy = (l1[1] + l2[1] + l3[1]) / 3
    return [ox, oy]

In [10]:
# Defining a Polygon from Shapely to get area and perimeter later
cent1 = centroid(afrUL, afrLT, afrUR)
cent2 = centroid(afrUL2, afrLT2, afrUR2)

poly1 = Polygon([tuple(afrUL), tuple(afrLT), tuple(afrUR)])
poly2 = Polygon([tuple(afrUL2), tuple(afrLT2), tuple(afrUR2)])

In [12]:
# Creating folium map of Africa with the estimated triangles estimated above overlaid
africa = folium.Map(location = cent2, zoom_start = 3.2)

folium.Polygon(locations = [tuple(afrUL), tuple(afrLT), tuple(afrUR)], 
               fill = True, color = 'Blue').add_to(africa)

folium.Polygon(locations = [tuple(afrUL2), tuple(afrLT2), tuple(afrUR2)], 
               fill = True, color = 'Red').add_to(africa)

folium.Marker(afrUL, popup = poly1.area, tooltip = "Area of Blue").add_to(africa)
folium.Marker(afrUR2, popup = poly2.area, tooltip = "Area of Red").add_to(africa)

africa

In [13]:
# Using the Shapely Polygon from above for area and perimeter
print("Area of Triangles")
print("Blue Triangle: ", poly1.area)
print("Red Triangle: ", poly2.area)
print("\n")
print("Perimeter of Triangles")
print("Blue Triangle: ", poly1.length)
print("Red Triangle: ", poly2.length)

Area of Triangles
Blue Triangle:  1460.4749244382713
Red Triangle:  1655.2288643486854


Perimeter of Triangles
Blue Triangle:  183.45068065878823
Red Triangle:  190.39939417579018


In [14]:
# PART 3

# Kansas Estimation
k1 = [39.976418758768894, -95.29692587953669]
k2 = [37.07662428131995, -94.63978519724449]
k3 = [36.995607797189386, -102.03808307555572]
k4 = [39.99756826032556, -102.04326155723051]

# Nebraska Estimation
n1 = [40.01087062060819, -102.04037945305414]
n2 = [41.00850302208278, -102.04651328270056]
n3 = [41.01872423220584, -104.02767748162101]
n4 = [42.98570034794942, -104.02739414921258]
n5 = [42.83425704034022, -97.25267596975529]
n6 = [40.00544669423327, -95.40806693464745]

In [15]:
kansas = Polygon([tuple(k1), tuple(k2), tuple(k3), tuple(k4)])
nebraska = Polygon([tuple(n1), tuple(n2), tuple(n3), tuple(n4), tuple(n5), tuple(n6)])

In [16]:
# Stores the Shapely objects into a Geo Series and saves it as a python feature collection
kansasGeo = gpd.GeoSeries([kansas]).__geo_interface__
nebraskaGeo = gpd.GeoSeries([nebraska]).__geo_interface__

In [17]:
# Save the feature collections into a geojson file
with open('kanNeb.geojson', 'w') as f:
   json.dump([kansasGeo, nebraskaGeo], f)

In [18]:
# Open the geojson file
with open('kanNeb.geojson', 'r') as f:
    dat = json.load(f)

In [19]:
# Function to read in the feature collections and turn them into polygon representations of Nebraska and Kansas used below
def geo_to_poly(df):
    poly1 = {}
    poly2 = {}
    for feature in df[0]['features']:
        for i in range(len(feature['geometry']['coordinates'][0])-1):
            poly1[i] = (feature['geometry']['coordinates'][0][i])
    for feature in df[1]['features']:
        for i in range(len(feature['geometry']['coordinates'][0])-1):
            poly2[i] = (feature['geometry']['coordinates'][0][i])
    return poly1, poly2

In [20]:
kanNeb = geo_to_poly(dat)

In [22]:
# Create folium map of our nebraska and kansas estimations
kanNebCenter = [40.180294336525115, -98.8762370625003]
kanNebMap = folium.Map(location = kanNebCenter, zoom_start = 6)

folium.Polygon(locations = [tuple(kanNeb[0][i] for i in range(len(kanNeb[0])))], 
               fill = True, color = 'Blue').add_to(kanNebMap)

folium.Polygon(locations = [tuple(kanNeb[1][i] for i in range(len(kanNeb[1])))], 
               fill = True, color = 'Red').add_to(kanNebMap)

kanNebMap

In [24]:
# PART 4

# Random Happiness Index for each state
happiness_index = np.random.randint(100, size = (50))
    
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [27]:
# Function to display certain text on the Choropleth Map
def happyText(nl):
    text = []
    for x in nl:
        if x >= 90:
            text.append('Very Happy')
        elif x < 90 and x >= 75:
            text.append('Pretty Happy')
        elif x < 75 and x >= 50:
            text.append('Could be Worse')
        elif x < 50 and x >= 25:
            text.append('This Sucks')
        else:
            text.append('Get Me Outta Here')
    return text

In [28]:
# Create Choropleth Map and plot our random happiness index for each state
fig = go.Figure(data=go.Choropleth(
    locations=states,
    z=happiness_index,
    locationmode='USA-states',
    colorscale='Greens',
    autocolorscale=False,
    text=happyText(happiness_index), # hover text
    marker_line_color='blue', # line markers between states
    colorbar_title="Happiness"
))

fig.update_layout(
    title_text='Happiness Index by State',
    title_x = .5,
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=False),
)

fig.show()

In [29]:
# PART 5

# Drawing randomly from the exponential distribution
from bokeh.plotting import figure, output_file, show

rand = np.random.exponential(scale = 2.5, size = 500)
hist, edges = np.histogram(rand, density = True, bins = 30)

In [32]:
# Plotting a Bokeh histogram from the randomly drawn data
expoHist = figure(title = "Random Exponential Distribution",
                 x_axis_label = "Value",
                 y_axis_label = "Counts")

expoHist.quad(top = hist, bottom = 0, left = edges[:-1], right = edges[1:], line_color = 'white')

# Plots an HTML of the Bokeh Histogram
show(expoHist)